In [45]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import manifold, datasets
import pickle
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import pairwise_distances
import pandas as pd
import seaborn as sns
from sklearn.covariance import MinCovDet
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import NearestNeighbors
from sklearn.svm import OneClassSVM
from sklearn.metrics import pairwise_distances
from sklearn.cluster import DBSCAN
from sklearn.linear_model import LinearRegression
import math
sys.path.append('..')
from lib.data_encoding import german_credit_encoding

In [87]:


class DetectorEnsemble:
    def __init__(self):
        self.detectors = []
        '''
        self.detectors.append(('iforest1', IsolationForest(random_state = 0, max_samples = 128, n_estimators = 100)))
        self.detectors.append(('iforest2', IsolationForest(random_state = 0, max_samples = 128, n_estimators = 200)))
        self.detectors.append(('iforest3', IsolationForest(random_state = 0, max_samples = 256, n_estimators = 100)))
        self.detectors.append(('iforest4', IsolationForest(random_state = 0, max_samples = 256, n_estimators = 200)))
        self.detectors.append(('iforest5', IsolationForest(random_state = 0, max_samples = 512, n_estimators = 100)))
        self.detectors.append(('iforest6', IsolationForest(random_state = 0, max_samples = 512, n_estimators = 200)))
        '''
        self.detectors.append(('knn', NearestNeighbors(algorithm='ball_tree')))
        self.detectors.append(('lof', LocalOutlierFactor(metric="precomputed")))
        #self.detectors.append(('robustcov', MinCovDet()))
        self.detectors.append(('iforest', IsolationForest()))
        self.detectors.append(('ocsvm', OneClassSVM()))
        self.detectors.append(('dbscan',  DBSCAN()))
    
    def fit_detector(self, X, y):
        self.clf = LinearRegression(fit_intercept=True, normalize=False, copy_X=True).fit(X, y)

    def fit(self, mat):
        dist = pairwise_distances(X = mat, metric='euclidean')
        self.scores = []
        for (name, detector) in self.detectors:
            if name[:3] == 'lof':
                detector.fit_predict(dist)
                self.scores.append(-detector.negative_outlier_factor_)
            elif name == 'robustcov':
                detector.fit(mat)
                self.scores.append(detector.mahalanobis(mat))
            elif name == 'knn':
                detector.fit(mat)
                self.scores.append(-detector.kneighbors(mat)[0][:, -1])
            elif name == 'dbscan':
                detector.fit(mat)
                score = np.array([1 if x == -1 else 0 for x in detector.labels_])
                self.scores.append(score)
            else:
                detector.fit_predict(mat)
                self.scores.append(-detector.score_samples(mat))
            print(name, min(self.scores[-1]), max(self.scores[-1]), self.scores[-1].shape)
        tmp = []
        for score in self.scores:
            min_s = np.min(score)
            max_s = np.max(score)
            range_s = max(1, max_s - min_s)
            score = (score - min_s) / range_s
            tmp.append(score)
        self.n = mat.shape[0]
        self.scores = np.array(tmp)
        self.ground_truth = {}
        self.adjust_sample_weight = self.n // 100
        self.weights = np.ones(len(self.detectors))
        weights = self.weights / np.sum(self.weights)

        self.scores = self.scores.transpose()
        y = (self.scores * weights).sum(axis = 1)
        print('before fit', self.scores.shape, y.shape)
        self.fit_detector(self.scores, y)
        print('after fit')
    
    def weighted_score(self):
        y = self.clf.predict(self.scores)
        for i in self.ground_truth:
            y[i] = self.ground_truth[i]
        return y

    def adjust_weight(self, idx, score):
        self.ground_truth[idx] = score
        sample_weight = np.ones(self.n)
        for i in self.ground_truth:
            sample_weight[i] = self.adjust_sample_weight
        y = self.weighted_score()
        self.fit_detector(self.scores, y)


model = pickle.load(open('../../output/dump/german0315v2.pkl', 'rb'))
paths = model['paths']
features = model['features']
mat = np.array([p['sample'] for p in paths]).astype('float')
for i in range(mat.shape[0]):
    mat[i] = mat[i] > 0
    mat[i] /= mat[i].sum() #np.sqrt(mat[i].sum())
all_dist = pairwise_distances(X = mat, metric='euclidean')


ensemble = DetectorEnsemble()
ensemble.fit(mat)
selected_path_idxes = ensemble.weighted_score().argsort()[::-1]

output_labels = ['reject', 'accept']

knn -0.9258200997725516 -0.07838233761296741 (10423,)
lof 0.9737859486100009 4.211928200115308 (10423,)
iforest 0.29394066764582155 0.3476671119984273 (10423,)
ocsvm -1421.6101449685095 -1.0811459656091529 (10423,)
dbscan 0 1 (10423,)
before fit (10423, 5) (10423,)
after fit


'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.


In [78]:
print(selected_path_idxes.tolist()[:50])

[4695, 4941, 8563, 2622, 7859, 9887, 6820, 9186, 3602, 1717, 476, 4272, 3016, 3733, 10158, 10067, 8246, 3206, 5387, 9237, 8978, 885, 4201, 9999, 1013, 1421, 10064, 10367, 8197, 7640, 7571, 2998, 9797, 8013, 10272, 7357, 333, 6030, 9776, 10374, 8802, 3325, 2844, 6374, 4891, 63, 8683, 3195, 6024, 127]


In [88]:
expected_count = 50
expected_one_class_count = 40
output_labels = ['reject', 'accept']

iforest_idxes = [4695, 4941, 8563, 2622, 7859, 9887, 6820, 9186, 3602, 1717, 476, 4272, 3016, 3733, 10158, 10067, 8246, 3206, 5387, 9237, 8978, 885, 4201, 9999, 1013, 1421, 10064, 10367, 8197, 7640, 7571, 2998, 9797, 8013, 10272, 7357, 333, 6030, 9776, 10374, 8802, 3325, 2844, 6374, 4891, 63, 8683, 3195, 6024, 127]
diff_idxes = [3719, 1572, 1224, 4632, 7869, 1788, 8489, 6267, 7082, 4721, 1032, 703, 6656, 10175, 5548, 10290, 6786, 1765, 7510, 2258, 3978, 71, 10180, 4910, 683, 1862, 1200, 5783, 4978, 4290, 2000, 185, 3032, 2324, 3306, 800, 802, 4821, 10297, 2247, 10029, 917, 3192, 2756, 847, 4706, 9480, 3682, 7524, 7616]
selected_path_idxes = diff_idxes + iforest_idxes

In [102]:
current_encoding = {
    'credit_risk' : ['No', 'Yes'], 
    'credit_history' : [
        "delay in paying off in the past",
        "critical account/other credits elsewhere",
        "no credits taken/all credits paid back duly",
        "existing credits paid back duly till now",
        "all credits at this bank paid back duly",
    ],
    'purpose' : [
        "others",
        "car (new)",
        "car (used)",
        "furniture/equipment",
        "radio/television",
        "domestic appliances",
        "repairs",
        "education",
        "vacation",
        "retraining",
        "business"
    ],
    'installment_rate': ["< 20", "20 <= ... < 25",  "25 <= ... < 35", ">= 35"],
    'present_residence': [
        "< 1 yr", 
        "1 <= ... < 4 yrs",
        "4 <= ... < 7 yrs", 
        ">= 7 yrs"
    ],
    'number_credits': ["1", "2~3", "4~5", ">= 6"],
    'people_liable': ["0 to 2", "3 or more"],
    'savings': [
        "unknown/no savings account",
        "... <  100 DM", 
        "100 <= ... <  500 DM",
        "500 <= ... < 1000 DM", 
        "... >= 1000 DM",
    ],
    'employment_duration': [
        "unemployed", 
        "< 1 yr", 
        "1 <= ... < 4 yrs",
        "4 <= ... < 7 yrs", 
        ">= 7 yrs"
    ],
    'personal_status_sex': [
        "not married male",
        "married male",
    ],
    'other_debtors': [
        'none',
        'co-applicant',
        'guarantor'
    ],
    'property': [
        "real estate",
        "building soc. savings agr./life insurance", 
        "car or other",
        "unknown / no property",
    ],
    'other_installment_plans': ['bank', 'stores', 'none'],
    'housing': ["rent", "own", "for free"],
    'job': [
        'unemployed/ unskilled - non-resident',
        'unskilled - resident',
        'skilled employee / official',
        'management/ self-employed/ highly qualified employee/ officer'
    ],
    'status': [
        "no checking account",
        "... < 0 DM",
        "0<= ... < 200 DM",
        "... >= 200 DM / salary for at least 1 year",
    ],
    'telephone': ['No', 'Yes'],
    'foreign_worker': ['No', 'Yes'],
}
rule_type = [
    1,1,1,1,1, 0,0,0,0,1, 
    0,0,0,1,1, 0,0,0,1,0,
    0,1,0,1,1, 0,0,0,0,1,
    1,1,0,0,0, 0,0,1,0,1,
    0,0,1,0,1, 0,0,1,0,1
] + [0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1]

mat2 = np.array([p['sample'] for p in paths]).astype('float')
for i in range(mat2.shape[0]):
    mat2[i] = mat2[i] > 0
    mat2[i] /= np.sqrt(mat2[i].sum())
all_dist = pairwise_distances(X = mat2, metric='cosine')
scores = ensemble.weighted_score()
score_with_idx = [(-x, i) for i, x in enumerate(scores)]
score_with_idx.sort()
rank = {}
for it, x in enumerate(score_with_idx):
    rank[x[1]] = it

anomaly_idxes = [x for i, x in enumerate(selected_path_idxes) if rule_type[i] > 0]
sample_counts = [(np.sum(paths[i]['sample']), i) for i in anomaly_idxes]
sample_counts.sort()
sample_counts = sample_counts[::-1]
anomaly_idxes = [x[1] for x in sample_counts]
sample_counts = [x[0] for x in sample_counts]

topk = 5
new_idxes = []
new_dists = []
new_pos = []
anomaly_idxes = [1379]
for it, idx in enumerate(anomaly_idxes):
    nearest = all_dist[idx, :].argsort()[:topk]
    dists = all_dist[idx, nearest]
    new_idxes += nearest.tolist()
    new_dists += dists.tolist()
    new_pos += [it] * topk

In [103]:

def interpret_path(path, features):
    conds = []
    for key in path['range']:
        feature = features[key]
        values = path['range'][key]
        name = feature['name']
        op = 'is'
        value = ''
        if feature['dtype'] == 'category':
            if len(values) < len(feature['values']) or key == 3:
                t_values = [1 if (i >= values[0] and i <= values[1]) else 0 for i in range(1, len(feature['values']) + 1)]
                values = t_values
            is_negation = np.sum(values) + 1 == len(values)
            if is_negation:
                op = 'is not'
                for i, d in enumerate(values):
                    if d == 0:
                        value = feature['values'][i]
                        break
            else:
                for i, d in enumerate(values):
                    if d == 1:
                        value = value + ' or ' + feature['values'][i]
                value = value[4:]
        else:
            op = 'in'
            value = '%d ~ %d' % (values[0], values[1])
        conds.append((name, op, value))
    output_label = output_labels[path['output']]
    # print(output_labels, path['output'])
    return conds, output_label

for index, feature in enumerate(features):
    if feature['name'] in current_encoding:
        feature['values'] = current_encoding[feature['name']]
    else:
        feature['values'] = feature['range']

rules = []
class_count = {}
max_n_conds = 0
for it, i in enumerate(new_idxes):
    conds, output = interpret_path(paths[i], features)
    #if class_count.get(output, 0) >= expected_one_class_count:
    #    continue
    class_count[output] = class_count.get(output, 0) + 1
    rules.append({'cond': conds, 'predict': output, 'index': i, 'dist': new_dists[it] })
    max_n_conds = max(len(conds), max_n_conds)
    #if len(rules) >= expected_count:
    #    break
conds_per_line = 4
max_n_conds = math.ceil(max_n_conds / conds_per_line) * conds_per_line


rule_idxes = [rule['index'] for rule in rules]

In [38]:
f = open('3.csv', 'w')

for it, rule in enumerate(rules):
    print(new_idxes[it])
    if it % topk == 0:
        s = '' + str(new_pos[it])
    else:
        s = 'dist: %.4f' % (new_dists[it])
    line = 0
    n_conds = len(rule['cond'])
    n_lines = math.ceil(n_conds / conds_per_line)
    base = it - it % 5
    overlap = np.sum(np.array(paths[rule['index']]['sample']) * np.array(paths[rules[base]['index']]['sample']))

    for line in range(n_lines):
        if line == 0:
            s += ',%d,%d,%d,IF,' % (rule['index'], np.sum(paths[rule['index']]['sample']), overlap)
        else:
            s += ',,,,,'
        for pos in range(conds_per_line):
            i = pos + line * conds_per_line
            if i < n_conds:
                item = rule['cond'][i]
                s += item[0] + ',' + item[1] + ',' + item[2] + ','
                s += 'AND,' if i < n_conds - 1 else ','
            else:
                s += '...,...,...,...,'
        if line == n_lines - 1:
            s += 'THEN,' + rule['predict']
        s += '\n'
    f.write(s + '\n')
f.close()

3719
3718
3720
3721
3717
1572
1571
5122
5123
526
1224
3331
3039
463
462
4632
4631
7241
4630
448
7869
8420
7868
4380
354
1032
1029
1033
4293
3021
10175
10174
10176
10179
10178
5548
5549
6710
6709
8741
7510
7509
7508
7507
7504
71
70
69
64
4321
10180
10181
6859
7338
3709
1862
1861
1860
1865
1867
2000
1997
1999
1998
2001
4290
4289
4291
3126
4288
185
186
184
1175
1178
4821
4822
4820
5836
4375


In [86]:
anomaly_idxes = [x for i, x in enumerate(selected_path_idxes) if rule_type[i] > 0]
print([np.sum(paths[i]['sample']) for i in anomaly_idxes])

[3, 2, 3, 2, 3, 3, 3, 3, 3, 3, 3, 2, 2, 3, 2, 2, 2, 2, 2, 2, 3, 62, 33, 50, 28, 28, 14, 30, 14, 65, 44, 30]


In [17]:
new_idxes[:5]

[3719, 372, 1378, 5010, 2324]

In [31]:
for it, i in enumerate(new_idxes):
    base = new_idxes[it - it % 5]
    print(np.sum(np.array(paths[i]['sample'])), np.sum(np.array(paths[i]['sample']) * np.array(paths[base]['sample'])))

3 3
1 1
1 1
6 2
2 1
2 2
3 1
13 2
4 1
17 2
3 3
2 1
2 1
9 2
3 1
2 2
10 2
14 2
15 2
16 2
3 3
1 1
1 1
1 1
5 2
3 3
1 1
2 1
2 1
9 2
3 3
14 3
7 2
7 2
11 2


In [98]:

nearest = all_dist[1379, :].argsort()[:topk]

32

In [83]:
print(np.max([np.sum(paths[i]['sample']) for i in selected_path_idxes]))

143


In [24]:
from model.german_rf import get_model
import sys
sys.path.append('../..')
from lib.tree_extractor import path_extractor
clf, (X_train, y_train, X_test, y_test, data_table), dataset, model, parameters = get_model()
paths = path_extractor(clf, 'random forest', (X_train, y_train))

target = 'credit_risk'
X = data_table.drop(target, axis=1).values
y = data_table[target].values
from lib.tree_extractor import assign_samples
assign_samples(paths, (X, y))

features = data_table.columns[1:]
new_feature = {}
feature_pos = {}
for index, feature in enumerate(features):
    if ' - ' in feature:
        name, p = feature.split(' - ')
        p = int(p)
        if name not in new_feature:
            new_feature[name] = []
        while p >= len(new_feature[name]):
            new_feature[name].append(-1)
        new_feature[name][p] = index
    else:
        new_feature[feature] = [index]

feature_range = {}
for key in new_feature:
    if key in data_table.columns:
        feature_range[key] = [data_table[key].min(), data_table[key].max() + 1]
    else:
        feature_range[key] = [0, len(new_feature[key])]
    for i, j in enumerate(new_feature[key]):
        feature_pos[j] = (key, i)


Test
Accuracy Score is 0.828
Precision Score is 0.8663101604278075
F1 Score is 0.88283378746594
Train
Accuracy Score is 0.8932692307692308


In [46]:

output_labels = ['reject', 'accept']
current_encoding = german_credit_encoding

def interpret_path(path):
    conds = {}
    for k in path['range']:
        name = feature_pos[k][0]
        val = path['range'][k]
        if name in current_encoding:
            if name not in conds:
                conds[name] = [1] * len(current_encoding[name])
            if name in data_table.columns:
                for i in range(feature_range[name][0], feature_range[name][1]):
                    if i < val[0] or i > val[1]:
                        conds[name][i - feature_range[name][0]] = 0
            else:
                if val[0] > 0:
                    conds[name] = [0] * len(current_encoding[name])
                    conds[name][feature_pos[k][1]] = 1
                else:
                    conds[name][feature_pos[k][1]] = 0
        else:
            cond = [max(feature_range[name][0], val[0]), min(feature_range[name][1], val[1])]
            conds[name] = cond

    output_conds = []
    for name in conds:
        val = conds[name]
        op = 'is'
        value = ''
        if name in current_encoding:
            is_negation = np.sum(val) + 1 == len(val) and len(val) > 2
            if is_negation:
                op = 'is not'
                for i, d in enumerate(val):
                    if d == 0:
                        value = current_encoding[name][i]
                        break
            else:
                for i, d in enumerate(val):
                    if d == 1:
                        value = value + ' or ' + current_encoding[name][i]
                value = value[4:]
        else:
            if val[0] == feature_range[name][0]:
                op = '<='
                value = int(val[1])
            elif val[1] == feature_range[name][1]:
                op = '>='
                value = int(val[0])
            else:
                op = 'in'
                value = '%d to %d' % (int(val[0]), int(val[1]))
        output_conds.append((name, op, value))
    output_label = output_labels[path['output']]
    # print(output_labels, path['output'])
    return conds, output_conds, output_label

In [48]:
interpret_path(paths[150])

({'savings': [1, 1, 0, 0, 0],
  'property': [0, 1, 1, 1],
  'purpose': [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'status': [1, 1, 0, 0],
  'housing': [0, 1, 1],
  'other_debtors': [1, 0, 0],
  'number_credits': [0, 0, 1, 1],
  'telephone': [1, 0]},
 [('savings', 'is', 'unknown/no savings account or ... <  100 DM'),
  ('property', 'is not', 'real estate'),
  ('purpose', 'is not', 'others'),
  ('status', 'is', 'no checking account or ... < 0 DM'),
  ('housing', 'is not', 'rent'),
  ('other_debtors', 'is', 'none'),
  ('number_credits', 'is', '4-5 or >= 6'),
  ('telephone', 'is', 'No')],
 'reject')